In [20]:
import pandas as pd
import numpy as np
datario_tx_precip_alerta_rio = pd.read_csv("database_csv/taxa_precipitacao_alertario.csv")

In [21]:
datario_tx_precip_alerta_rio.shape

(9710540, 9)

In [22]:
# Concatenar 'data_particao' e 'horario'
datario_tx_precip_alerta_rio['data_hora'] = datario_tx_precip_alerta_rio['data_particao'] + ' ' + datario_tx_precip_alerta_rio['horario']

# Converter para datetime
datario_tx_precip_alerta_rio['data_hora'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_hora'])
datario_tx_precip_alerta_rio['data_particao'] = pd.to_datetime(datario_tx_precip_alerta_rio['data_particao'])


datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio.sort_values(by='data_hora', ascending=True)




In [23]:
num_NaT = datario_tx_precip_alerta_rio['data_hora'].isna().sum()
num_NaT


2761

In [24]:
datario_tx_precip_alerta_rio = datario_tx_precip_alerta_rio[~pd.isna(datario_tx_precip_alerta_rio["data_hora"])]

In [25]:
# Preencher valores NaN com 0 nas colunas especificadas
colunas_chuva = ['acumulado_chuva_15_min', 'acumulado_chuva_1_h', 'acumulado_chuva_4_h', 'acumulado_chuva_24_h']

datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].fillna(0)
datario_tx_precip_alerta_rio[colunas_chuva] = datario_tx_precip_alerta_rio[colunas_chuva].astype(float)

In [26]:
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio.loc[datario_tx_precip_alerta_rio['data_particao'].dt.year >= 2016].copy()

datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_hora
4324741,4_2016-01-01 00:00:00,4,0.0,0.0,0.0,2.4,3.4,00:00:00,2016-01-01,2016-01-01 00:00:00
3013996,32_2016-01-01 00:00:00,32,0.0,0.0,0.0,0.8,2.6,00:00:00,2016-01-01,2016-01-01 00:00:00
1782759,31_2016-01-01 00:00:00,31,0.0,0.0,0.0,13.6,23.8,00:00:00,2016-01-01,2016-01-01 00:00:00
4944657,5_2016-01-01 00:00:00,5,0.0,0.0,0.0,22.2,30.4,00:00:00,2016-01-01,2016-01-01 00:00:00
7887435,33_2016-01-01 00:00:00,33,0.0,0.0,0.0,0.2,3.2,00:00:00,2016-01-01,2016-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...
8012624,7_2024-03-22 14:00:00,7,1.8,9.6,15.8,16.4,16.4,14:00:00,2024-03-22,2024-03-22 14:00:00
2480671,3_2024-03-22 14:00:00,3,3.8,7.2,14.6,23.6,23.6,14:00:00,2024-03-22,2024-03-22 14:00:00
8864160,20_2024-03-22 14:00:00,20,0.2,0.8,6.6,7.2,7.2,14:00:00,2024-03-22,2024-03-22 14:00:00
7139761,25_2024-03-22 14:00:00,25,0.6,2.6,9.8,14.8,14.8,14:00:00,2024-03-22,2024-03-22 14:00:00


In [27]:
datario_tx_precip_alerta_rio_red.dtypes

primary_key                       object
id_estacao                         int64
acumulado_chuva_15_min           float64
acumulado_chuva_1_h              float64
acumulado_chuva_4_h              float64
acumulado_chuva_24_h             float64
acumulado_chuva_96_h             float64
horario                           object
data_particao             datetime64[ns]
data_hora                 datetime64[ns]
dtype: object

In [28]:
# # grupo_estacao = datario_tx_precip_alerta_rio_red.groupby('id_estacao')
# datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red[datario_tx_precip_alerta_rio_red["id_estacao"] == 1].copy().reset_index(drop = True)
# # Calculando o acumulado de 45 minutos para cada estação
# datario_tx_precip_alerta_rio_red['acumulado_45min'] = datario_tx_precip_alerta_rio_red['acumulado_chuva_15_min'].rolling(window=3).sum()


# datario_tx_precip_alerta_rio_red

In [32]:
# Definir a função criar_evento_chuvoso
def criar_evento_chuvoso(df):
    # Calculate if each row is the start of an event
    df['evento_inicio'] = (df['acumulado_chuva_480_min'] != 0) & (df['acumulado_chuva_480_min'].shift(1) == 0)
    
    # Create a series to represent the event numbers
    evento_nums = df['evento_inicio'].cumsum()
    
    # Create the event IDs
    df['evento_chuvoso'] = df['id_estacao'].astype(str) + '_evento_' + evento_nums.map(str)
    
    # Filter out non-event rows
    df['evento_chuvoso'] = np.where(df['acumulado_chuva_480_min'] == 0, np.nan, df['evento_chuvoso'])
    
    # Drop intermediate columns
    df.drop(columns=['evento_inicio'], inplace=True)
    
    return df

In [33]:
# Lista para armazenar os DataFrames modificados para cada estação
dfs_rol_sum = []
estacoes = datario_tx_precip_alerta_rio["id_estacao"].unique().tolist()

# Loop sobre cada id_estacao único no DataFrame original
for estacao in estacoes:
    # Filtrar o DataFrame para a estação atual
    df_estacao = datario_tx_precip_alerta_rio_red[datario_tx_precip_alerta_rio_red['id_estacao'] == estacao].copy()
    
    # Lista para armazenar os dicionários de mapeamento para cada coluna
    mapping_dicts = []
    
    # Loop sobre cada coluna de acumulado de chuva com diferentes intervalos de tempo
    for intervalo in [30, 45, 90, 120, 360, 480, 720]:
        ## Criando o loop para as colunas de minutos

        # Criar a janela para a soma móvel
        window_num = int(intervalo/15)

        # Aplicar a operação de rolling sum na coluna atual
        df_estacao[f'acumulado_chuva_{intervalo}_min'] = df_estacao["acumulado_chuva_15_min"].rolling(window=window_num).sum()
        
        # Mapear os resultados de volta ao DataFrame original usando o primary_key
        mapping_dict = df_estacao.set_index('primary_key')[f'acumulado_chuva_{intervalo}_min'].to_dict()
        mapping_dicts.append(mapping_dict)

    df_estacao.fillna(0,inplace = True)
    # Adicionar o DataFrame modificado à lista
    dfs_rol_sum.append(df_estacao)


dfs_rol_sum = list(map(criar_evento_chuvoso, dfs_rol_sum))
    
# Concatenar todos os DataFrames modificados de volta em um único DataFrame
datario_tx_precip_alerta_rio_red = pd.concat(dfs_rol_sum)


In [34]:
datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red.sort_values(by='data_hora', ascending=True)
datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,horario,data_particao,data_hora,acumulado_chuva_30_min,acumulado_chuva_45_min,acumulado_chuva_90_min,acumulado_chuva_120_min,acumulado_chuva_360_min,acumulado_chuva_480_min,acumulado_chuva_720_min,evento_chuvoso
4944657,5_2016-01-01 00:00:00,5,0.0,0.0,0.0,22.2,30.4,00:00:00,2016-01-01,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3013996,32_2016-01-01 00:00:00,32,0.0,0.0,0.0,0.8,2.6,00:00:00,2016-01-01,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7887435,33_2016-01-01 00:00:00,33,0.0,0.0,0.0,0.2,3.2,00:00:00,2016-01-01,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8534335,2_2016-01-01 00:00:00,2,0.0,0.0,0.0,8.2,18.0,00:00:00,2016-01-01,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1782759,31_2016-01-01 00:00:00,31,0.0,0.0,0.0,13.6,23.8,00:00:00,2016-01-01,2016-01-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138514,29_2024-03-22 14:00:00,29,0.2,1.8,3.0,20.4,20.4,14:00:00,2024-03-22,2024-03-22 14:00:00,0.8,1.2,2.0,2.2,2.6,2.6,3.4,29_evento_868
8864160,20_2024-03-22 14:00:00,20,0.2,0.8,6.6,7.2,7.2,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.4,1.0,1.4,12.6,15.0,16.0,20_evento_859
7022422,19_2024-03-22 14:00:00,19,0.0,3.6,17.0,24.2,24.8,14:00:00,2024-03-22,2024-03-22 14:00:00,0.2,0.6,5.2,7.2,41.2,43.4,56.4,19_evento_873
2512434,5_2024-03-22 14:00:00,5,1.6,9.6,21.6,26.6,27.4,14:00:00,2024-03-22,2024-03-22 14:00:00,3.8,6.6,16.6,19.4,34.4,38.4,53.6,5_evento_836


In [35]:
datario_tx_precip_alerta_rio_red["data_hora"].unique()

array(['2016-01-01T00:00:00.000000000', '2016-01-01T00:15:00.000000000',
       '2016-01-01T00:30:00.000000000', ...,
       '2024-03-22T13:50:00.000000000', '2024-03-22T13:55:00.000000000',
       '2024-03-22T14:00:00.000000000'], dtype='datetime64[ns]')

In [36]:
datario_tx_precip_alerta_rio_red.dropna(inplace=True)
datario_tx_precip_alerta_rio_red.reset_index(drop=True, inplace=True)

In [38]:
# Puro suco dos dados
colunas_ordenadas = [
    'primary_key',
    'id_estacao',
    'acumulado_chuva_15_min',
    'acumulado_chuva_30_min',
    'acumulado_chuva_45_min',
    'acumulado_chuva_90_min',
    'acumulado_chuva_120_min',
    'acumulado_chuva_1_h',
    'acumulado_chuva_4_h',
    'acumulado_chuva_24_h',
    'acumulado_chuva_96_h',
    'acumulado_chuva_360_min',
    'acumulado_chuva_480_min',
    'acumulado_chuva_720_min',
    'horario',
    'data_particao',
    'data_hora',
    'evento_chuvoso'
]

datario_tx_precip_alerta_rio_red = datario_tx_precip_alerta_rio_red[colunas_ordenadas]
datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_30_min,acumulado_chuva_45_min,acumulado_chuva_90_min,acumulado_chuva_120_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,acumulado_chuva_360_min,acumulado_chuva_480_min,acumulado_chuva_720_min,horario,data_particao,data_hora,evento_chuvoso
0,33_2016-01-01 18:00:00,33,0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.2,3.4,0.2,0.2,0.2,18:00:00,2016-01-01,2016-01-01 18:00:00,33_evento_1
1,33_2016-01-01 18:15:00,33,0.0,0.2,0.2,0.2,0.2,0.2,0.2,0.2,3.4,0.2,0.2,0.2,18:15:00,2016-01-01,2016-01-01 18:15:00,33_evento_1
2,33_2016-01-01 18:30:00,33,0.0,0.0,0.2,0.2,0.2,0.2,0.2,0.2,3.4,0.2,0.2,0.2,18:30:00,2016-01-01,2016-01-01 18:30:00,33_evento_1
3,33_2016-01-01 18:45:00,33,0.0,0.0,0.0,0.2,0.2,0.2,0.2,0.2,3.4,0.2,0.2,0.2,18:45:00,2016-01-01,2016-01-01 18:45:00,33_evento_1
4,33_2016-01-01 19:00:00,33,0.0,0.0,0.0,0.2,0.2,0.0,0.2,0.2,3.4,0.2,0.2,0.2,19:00:00,2016-01-01,2016-01-01 19:00:00,33_evento_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807546,29_2024-03-22 14:00:00,29,0.2,0.8,1.2,2.0,2.2,1.8,3.0,20.4,20.4,2.6,2.6,3.4,14:00:00,2024-03-22,2024-03-22 14:00:00,29_evento_868
1807547,20_2024-03-22 14:00:00,20,0.2,0.2,0.4,1.0,1.4,0.8,6.6,7.2,7.2,12.6,15.0,16.0,14:00:00,2024-03-22,2024-03-22 14:00:00,20_evento_859
1807548,19_2024-03-22 14:00:00,19,0.0,0.2,0.6,5.2,7.2,3.6,17.0,24.2,24.8,41.2,43.4,56.4,14:00:00,2024-03-22,2024-03-22 14:00:00,19_evento_873
1807549,5_2024-03-22 14:00:00,5,1.6,3.8,6.6,16.6,19.4,9.6,21.6,26.6,27.4,34.4,38.4,53.6,14:00:00,2024-03-22,2024-03-22 14:00:00,5_evento_836


In [41]:
# Selecionando os eventos chuvosos

colunas_tempo = ['acumulado_chuva_15_min',
    'acumulado_chuva_30_min',
    'acumulado_chuva_45_min',
    'acumulado_chuva_90_min',
    'acumulado_chuva_120_min',
    'acumulado_chuva_1_h',
    'acumulado_chuva_4_h',
    'acumulado_chuva_24_h',
    'acumulado_chuva_96_h',
    'acumulado_chuva_360_min',
    'acumulado_chuva_480_min',
    'acumulado_chuva_720_min']

for element in colunas_tempo:
    if element[-1] == 'h':
        horas = int(element.split('_')[-2])
        datario_tx_precip_alerta_rio_red[element] = datario_tx_precip_alerta_rio_red[element]/horas
    else:
        minutos_h = int(element.split('_')[-2])/60
        datario_tx_precip_alerta_rio_red[element] = datario_tx_precip_alerta_rio_red[element]/minutos_h



datario_tx_precip_alerta_rio_red

,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_30_min,acumulado_chuva_45_min,acumulado_chuva_90_min,acumulado_chuva_120_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,acumulado_chuva_360_min,acumulado_chuva_480_min,acumulado_chuva_720_min,horario,data_particao,data_hora,evento_chuvoso
0,33_2016-01-01 18:00:00,33,0.8,0.4,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.035417,0.033333,0.025,0.016667,18:00:00,2016-01-01,2016-01-01 18:00:00,33_evento_1
1,33_2016-01-01 18:15:00,33,0.0,0.4,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.035417,0.033333,0.025,0.016667,18:15:00,2016-01-01,2016-01-01 18:15:00,33_evento_1
2,33_2016-01-01 18:30:00,33,0.0,0.0,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.035417,0.033333,0.025,0.016667,18:30:00,2016-01-01,2016-01-01 18:30:00,33_evento_1
3,33_2016-01-01 18:45:00,33,0.0,0.0,0.000000,0.133333,0.1,0.2,0.05,0.008333,0.035417,0.033333,0.025,0.016667,18:45:00,2016-01-01,2016-01-01 18:45:00,33_evento_1
4,33_2016-01-01 19:00:00,33,0.0,0.0,0.000000,0.133333,0.1,0.0,0.05,0.008333,0.035417,0.033333,0.025,0.016667,19:00:00,2016-01-01,2016-01-01 19:00:00,33_evento_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807546,29_2024-03-22 14:00:00,29,0.8,1.6,1.600000,1.333333,1.1,1.8,0.75,0.850000,0.212500,0.433333,0.325,0.283333,14:00:00,2024-03-22,2024-03-22 14:00:00,29_evento_868
1807547,20_2024-03-22 14:00:00,20,0.8,0.4,0.533333,0.666667,0.7,0.8,1.65,0.300000,0.075000,2.100000,1.875,1.333333,14:00:00,2024-03-22,2024-03-22 14:00:00,20_evento_859
1807548,19_2024-03-22 14:00:00,19,0.0,0.4,0.800000,3.466667,3.6,3.6,4.25,1.008333,0.258333,6.866667,5.425,4.700000,14:00:00,2024-03-22,2024-03-22 14:00:00,19_evento_873
1807549,5_2024-03-22 14:00:00,5,6.4,7.6,8.800000,11.066667,9.7,9.6,5.40,1.108333,0.285417,5.733333,4.800,4.466667,14:00:00,2024-03-22,2024-03-22 14:00:00,5_evento_836


In [54]:
def chuvas_mais_fortes(df, coluna_tempo, data_inicio=None, data_fim=None, quantidade_eventos=10, id_estacao=None):
    # Filtrar o DataFrame pelas datas de início e fim, se fornecidas
    if data_inicio is not None and data_fim is not None:
        df = df[(df['data_particao'] >= data_inicio) & (df['data_particao'] <= data_fim)]
    elif data_inicio is not None:
        df = df[df['data_particao'] >= data_inicio]
    elif data_fim is not None:
        df = df[df['data_particao'] <= data_fim]
    
    # Filtrar o DataFrame pela estação, se fornecido
    if id_estacao is not None:
        df = df[df['id_estacao'] == id_estacao]
    
     # Ordenar o DataFrame pela coluna de tempo especificada
    df_ordenado = df.sort_values(by=coluna_tempo, ascending=False)
    
    # Retornar os x primeiros elementos diferentes da coluna 'evento_chuvoso'
    chuvas_fortes = df_ordenado['evento_chuvoso'].drop_duplicates().head(quantidade_eventos).tolist()
    
    # Retornar o DataFrame filtrado apenas com esses eventos
    df_eventos = df[df['evento_chuvoso'].isin(chuvas_fortes)]


    return chuvas_fortes, df_eventos

resultados_eventos, df_eventos = chuvas_mais_fortes(datario_tx_precip_alerta_rio_red, 'acumulado_chuva_1_h', quantidade_eventos=10, id_estacao=8)

Eventos: ['8_evento_827', '8_evento_418', '8_evento_214', '8_evento_488', '8_evento_520', '8_evento_726', '8_evento_812', '8_evento_820', '8_evento_33', '8_evento_329']

DataFrame com eventos filtrados:


,primary_key,id_estacao,acumulado_chuva_15_min,acumulado_chuva_30_min,acumulado_chuva_45_min,acumulado_chuva_90_min,acumulado_chuva_120_min,acumulado_chuva_1_h,acumulado_chuva_4_h,acumulado_chuva_24_h,acumulado_chuva_96_h,acumulado_chuva_360_min,acumulado_chuva_480_min,acumulado_chuva_720_min,horario,data_particao,data_hora,evento_chuvoso
1782801,8_2024-02-17 11:30:00,8,2353.6,2226.0,1569.066667,1091.733333,1049.2,1049.2,262.30,43.716667,11.593750,349.733333,262.300,174.866667,11:30:00,2024-02-17,2024-02-17 11:30:00,8_evento_827
1782804,8_2024-02-17 11:45:00,8,0.0,127.6,784.533333,1176.800000,882.6,1049.2,262.30,43.716667,11.593750,447.800000,335.850,223.900000,11:45:00,2024-02-17,2024-02-17 11:45:00,8_evento_827
1782809,8_2024-02-17 12:10:00,8,0.0,0.0,0.000000,0.000000,294.2,588.4,262.30,43.716667,11.593750,447.800000,335.850,223.900000,12:10:00,2024-02-17,2024-02-17 12:10:00,8_evento_827
1782808,8_2024-02-17 12:05:00,8,0.0,0.0,0.000000,42.533333,588.4,588.4,262.30,43.716667,11.593750,447.800000,335.850,223.900000,12:05:00,2024-02-17,2024-02-17 12:05:00,8_evento_827
1782807,8_2024-02-17 12:00:00,8,0.0,0.0,0.000000,392.266667,850.7,0.0,262.30,43.716667,11.593750,447.800000,335.850,223.900000,12:00:00,2024-02-17,2024-02-17 12:00:00,8_evento_827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048642,8_2020-12-06 14:00:00,8,0.0,0.0,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.004167,0.033333,0.025,0.016667,14:00:00,2020-12-06,2020-12-06 14:00:00,8_evento_488
1048616,8_2020-12-06 13:45:00,8,0.0,0.4,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.004167,0.033333,0.025,0.016667,13:45:00,2020-12-06,2020-12-06 13:45:00,8_evento_488
1048594,8_2020-12-06 13:30:00,8,0.8,0.4,0.266667,0.133333,0.1,0.2,0.05,0.008333,0.004167,0.033333,0.025,0.016667,13:30:00,2020-12-06,2020-12-06 13:30:00,8_evento_488
1783346,8_2024-02-20 16:40:00,8,0.8,0.8,0.533333,0.266667,0.2,0.2,0.05,0.008333,0.002083,447.866667,335.900,223.933333,16:40:00,2024-02-20,2024-02-20 16:40:00,8_evento_827
